In [12]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
import tensorflowjs as tfjs
import json
import os

In [13]:
# ==================== 1. ĐỌC DỮ LIỆU ====================
print("Đang đọc data.csv...")
df = pd.read_csv("data.csv")
df.columns = df.columns.str.strip()

# Tạo đặc trưng
df['AvgBPM'] = df['PULSE']
df['A_total'] = df['RESP'] * 0.5
df = df.dropna(subset=['AvgBPM', 'A_total'])

Đang đọc data.csv...


In [14]:
# ==================== TẠO DỮ LIỆU GIẢ LẬP ====================
print("Tạo dữ liệu giả lập đủ 8 lớp...")
np.random.seed(42)
fake_rows = []
for _ in range(50):
    fake_rows.append([94, 93, 20, 97, 93, 10.0, 'Bình thường'])
    fake_rows.append([95, 94, 30, 97, 94, 15.0, 'Hoạt động nhẹ'])
    fake_rows.append([96, 95, 50, 96, 95, 25.0, 'Hoạt động trung bình'])
    fake_rows.append([97, 96, 80, 95, 96, 40.0, 'Hoạt động mạnh'])
    fake_rows.append([55, 54, 20, 96, 54, 10.0, 'Cảnh báo sức khỏe không ổn định'])
    fake_rows.append([130, 129, 20, 95, 129, 10.0, 'Nhịp tim cao bất thường'])
    fake_rows.append([45, 44, 20, 99, 44, 10.0, 'Sức khỏe siêu tốt'])
    fake_rows.append([160, 158, 80, 90, 158, 40.0, 'Cảnh báo nguy hiểm'])

fake_df = pd.DataFrame(fake_rows, columns=['HR', 'PULSE', 'RESP', 'SpO2', 'AvgBPM', 'A_total', 'label'])
df = pd.concat([df, fake_df], ignore_index=True)

Tạo dữ liệu giả lập đủ 8 lớp...


In [15]:
# ==================== 3. TẠO NHÃN ====================
def classify_health_state(avg_bpm, a_total):
    if avg_bpm > 150 and a_total > 35: return 'Cảnh báo nguy hiểm'
    if avg_bpm < 50 and a_total < 12: return 'Sức khỏe siêu tốt'
    if avg_bpm > 110:
        if a_total < 12: return 'Nhịp tim cao bất thường'
        else: return 'Hoạt động mạnh'
    if avg_bpm < 60:
        if a_total < 12: return 'Cảnh báo sức khỏe không ổn định'
        else: return 'Hoạt động mạnh'
    if 60 <= avg_bpm <= 110:
        if a_total < 12: return 'Bình thường'
        elif a_total < 20: return 'Hoạt động nhẹ'
        elif a_total < 35: return 'Hoạt động trung bình'
        else: return 'Hoạt động mạnh'
    return 'Không xác định'

df['label'] = df.apply(lambda row: classify_health_state(row['AvgBPM'], row['A_total']), axis=1)
df = df[df['label'] != 'Không xác định']

print("Phân bố nhãn:")
print(df['label'].value_counts())

Phân bố nhãn:
label
Bình thường                        477
Hoạt động nhẹ                       91
Hoạt động trung bình                50
Hoạt động mạnh                      50
Cảnh báo sức khỏe không ổn định     50
Nhịp tim cao bất thường             50
Sức khỏe siêu tốt                   50
Cảnh báo nguy hiểm                  50
Name: count, dtype: int64


In [16]:
# ==================== 4. CHUẨN BỊ DỮ LIỆU ====================
X = df[["AvgBPM", "A_total"]].values
y = df["label"].values

# Encode label
desired_labels = ['Bình thường','Hoạt động nhẹ','Hoạt động trung bình','Hoạt động mạnh',
                  'Cảnh báo sức khỏe không ổn định','Nhịp tim cao bất thường','Sức khỏe siêu tốt','Cảnh báo nguy hiểm']
label_map = {label: idx for idx, label in enumerate(desired_labels)}
y_encoded = np.array([label_map[label] for label in y])

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded)

In [17]:
# ==================== 5. MODEL ====================
model = keras.Sequential([
    keras.layers.Dense(16, activation='relu', input_shape=(2,)),
    keras.layers.Dropout(0.3),
    keras.layers.Dense(8, activation='relu'),
    keras.layers.Dropout(0.3),
    keras.layers.Dense(8, activation='softmax')
])
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=100, validation_data=(X_test, y_test), verbose=1)

Epoch 1/100
22/22 [==============================] - 1s 13ms/step - loss: 2.0705 - accuracy: 0.1816 - val_loss: 2.0131 - val_accuracy: 0.6092
Epoch 2/100
22/22 [==============================] - 0s 4ms/step - loss: 2.0035 - accuracy: 0.4280 - val_loss: 1.9478 - val_accuracy: 0.6667
Epoch 3/100
22/22 [==============================] - 0s 3ms/step - loss: 1.9311 - accuracy: 0.5706 - val_loss: 1.8805 - val_accuracy: 0.6667
Epoch 4/100
22/22 [==============================] - 0s 3ms/step - loss: 1.8719 - accuracy: 0.6138 - val_loss: 1.8047 - val_accuracy: 0.6667
Epoch 5/100
22/22 [==============================] - 0s 5ms/step - loss: 1.7905 - accuracy: 0.6153 - val_loss: 1.7152 - val_accuracy: 0.6667
Epoch 6/100
22/22 [==============================] - 0s 5ms/step - loss: 1.6954 - accuracy: 0.6196 - val_loss: 1.6113 - val_accuracy: 0.6092
Epoch 7/100
22/22 [==============================] - 0s 4ms/step - loss: 1.6012 - accuracy: 0.6167 - val_loss: 1.4896 - val_accuracy: 0.6092
Epoch 8/100


In [18]:
# ==================== 6. ĐÁNH GIÁ ====================
y_pred = np.argmax(model.predict(X_test), axis=1)
unique_labels = np.unique(y_test)
label_names = [desired_labels[i] for i in unique_labels]
print(classification_report(y_test, y_pred, labels=unique_labels, target_names=label_names))

6/6 [==============================] - 0s 1ms/step
                                 precision    recall  f1-score   support

                    Bình thường       0.90      1.00      0.95        96
                  Hoạt động nhẹ       1.00      0.39      0.56        18
           Hoạt động trung bình       1.00      1.00      1.00        10
                 Hoạt động mạnh       1.00      1.00      1.00        10
Cảnh báo sức khỏe không ổn định       0.00      0.00      0.00        10
        Nhịp tim cao bất thường       1.00      1.00      1.00        10
              Sức khỏe siêu tốt       0.50      1.00      0.67        10
             Cảnh báo nguy hiểm       1.00      1.00      1.00        10

                       accuracy                           0.88       174
                      macro avg       0.80      0.80      0.77       174
                   weighted avg       0.86      0.88      0.85       174



c:\Users\ACER\Anaconda\envs\XuLyAnh\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\ACER\Anaconda\envs\XuLyAnh\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\ACER\Anaconda\envs\XuLyAnh\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [19]:
# ==================== 7. CONVERT ====================
os.makedirs('tfjs_model', exist_ok=True)
tfjs.converters.save_keras_model(model, 'tfjs_model')

with open('scaler.json', 'w') as f:
    json.dump({'mean': scaler.mean_.tolist(), 'scale': scaler.scale_.tolist()}, f, indent=4)

print("HOÀN TẤT! File tfjs_model/ + scaler.json đã tạo")

HOÀN TẤT! File tfjs_model/ + scaler.json đã tạo
